<a href="https://colab.research.google.com/github/DiwakarBasnet/Chatbot/blob/main/generative_chatbot_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.1 MB/s eta 0:00:00


# Load Model

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_checkpoint = 'microsoft/DialoGPT-small'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
# Let's chat for 5 lines
for step in range(5):
  # encode the new user input, add the eos_token and return a tensor in PyTorch
  new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

  # append the new user input tokens to the chat history
  bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

  # generated a response while limiting the total chat history to 1000 tokens
  chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id = tokenizer.eos_token_id)

  # pretty print last output tokens from bot
  print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:hello


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Hi
>> User:how you doning?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: How you doin '?
>> User:shut up


...

DialoGPT: I'm not sure what you're trying to say.
>> User:dana shut up


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure what you're trying to say.
>> User:i am trying to tell you to shut up


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure what you're trying to say.


# Load Data

In [3]:
import pandas as pd

theovon_data = pd.read_csv('/content/drive/MyDrive/YouTube-Data/TheoVon/TheoVon-transcripts-v4.csv')
print('Length of data: ', len(theovon_data))
theovon_data.head()

Length of data:  485534


,episode,time,text
0,3,0.0,[Music] you know what that is don't you
1,3,24.27,you know what that is it's a little house on t...
2,3,28.349,the Daniel Caine Orchestra remember that let's...
3,3,41.82,dirty trick was it trumpet a horn that's it th...
4,3,51.469,the Prairie theme song God that was a good one...


In [4]:
# Eliminate the \n in text
theovon_data['text'] = theovon_data['text'].str.replace('\n', ' ')

In [5]:
contexted = []
n = 7
text = theovon_data['text']

for i in range(n, len(text)):
  row = []
  prev = i - 1 - n
  for j in range(i, prev, -1):
    row.append(text[j])
  contexted.append(row)
contexted[:5]

[["it's 12:40 a.m. I had a show tonight at the Hollywood improv and afterwards I",
  'brah I bring this to you it is January 16th',
  "time some people know that some people don't now you know it Michael Landon",
  'the Prairie theme song God that was a good one one of my favorite shows of all',
  "dirty trick was it trumpet a horn that's it that is it the Little House on",
  "the Daniel Caine Orchestra remember that let's hear that intro again dude that",
  "you know what that is it's a little house on the prairie theme song baby",
  "[Music] you know what that is don't you"],
 ["had a fan of my friend I say a friend a lady that I've met through comedy",
  "it's 12:40 a.m. I had a show tonight at the Hollywood improv and afterwards I",
  'brah I bring this to you it is January 16th',
  "time some people know that some people don't now you know it Michael Landon",
  'the Prairie theme song God that was a good one one of my favorite shows of all',
  "dirty trick was it trumpet a horn th

In [6]:
columns = ['response', 'context'] + ['context/'+str(i) for i in range(n-1)]
columns

['response',
 'context',
 'context/0',
 'context/1',
 'context/2',
 'context/3',
 'context/4',
 'context/5']

In [7]:
df = pd.DataFrame.from_records(contexted, columns=columns)
df.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,it's 12:40 a.m. I had a show tonight at the Ho...,brah I bring this to you it is January 16th,time some people know that some people don't n...,the Prairie theme song God that was a good one...,dirty trick was it trumpet a horn that's it th...,the Daniel Caine Orchestra remember that let's...,you know what that is it's a little house on t...,[Music] you know what that is don't you
1,had a fan of my friend I say a friend a lady t...,it's 12:40 a.m. I had a show tonight at the Ho...,brah I bring this to you it is January 16th,time some people know that some people don't n...,the Prairie theme song God that was a good one...,dirty trick was it trumpet a horn that's it th...,the Daniel Caine Orchestra remember that let's...,you know what that is it's a little house on t...
2,brought me two shirts as a gift one of them ha...,had a fan of my friend I say a friend a lady t...,it's 12:40 a.m. I had a show tonight at the Ho...,brah I bring this to you it is January 16th,time some people know that some people don't n...,the Prairie theme song God that was a good one...,dirty trick was it trumpet a horn that's it th...,the Daniel Caine Orchestra remember that let's...
3,Landon on the front Oh beautiful shirt dude so...,brought me two shirts as a gift one of them ha...,had a fan of my friend I say a friend a lady t...,it's 12:40 a.m. I had a show tonight at the Ho...,brah I bring this to you it is January 16th,time some people know that some people don't n...,the Prairie theme song God that was a good one...,dirty trick was it trumpet a horn that's it th...
4,something and another shirt it said all day I ...,Landon on the front Oh beautiful shirt dude so...,brought me two shirts as a gift one of them ha...,had a fan of my friend I say a friend a lady t...,it's 12:40 a.m. I had a show tonight at the Ho...,brah I bring this to you it is January 16th,time some people know that some people don't n...,the Prairie theme song God that was a good one...


In [8]:
from sklearn.model_selection import train_test_split
trn_df, val_df = train_test_split(df, test_size=0.1)
print("Length of Training data: ", len(trn_df))
print("Length of val data: ", len(val_df))

Length of Training data:  436974
Length of val data:  48553


In [9]:
trn_df = trn_df.sample(25000)
val_df = val_df.sample(2000)

In [10]:
trn_df[:3]

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
48800,was from Branson I mean this dude was getting ...,had like six gallons or something somebody you...,couple of buckets come down from from Missouri...,getting water from Georgia he was getting wate...,you were 17 so I see Mike you know he was gett...,neighborhood you know you were lucky to get to...,his house and eat the ice because a lot of us ...,America and we were good and he would freeze i...
416788,like four or five hundred views i feel a littl...,really yeah if it's one view i'm okay with it ...,i'm looking at porno video that's why i look a...,watching the same time that's the part that se...,video with porno video it seems like there's 7...,then you it was your thing now when you're sit...,but the crazy thing was it used to feel like i...,right there's many options outside of animalia
450156,50 cents he's always on it yeah yeah man come ...,like oh [ __ ] all the time like Lil Wayne's d...,call them punks yeah you know well all the rap...,in the black community they're not as acceptin...,Obama legalized there right probably yeah it w...,changed I mean what was it not when did it bec...,gay being gay used to be a secret it was like ...,dude I remember we didn't you know they didn't...


In [11]:
import os
import pickle
import logging
from transformers import PreTrainedTokenizer, MODEL_WITH_LM_HEAD_MAPPING, WEIGHTS_NAME
from torch.nn.utils.rnn import pad_sequence   # Pad a list of variable length Tensors with paddin_value
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pathlib import Path
from typing import Dict, List, Tuple

logger = logging.getLogger(__name__)

# Function to convert data into format our model can understand
def construct_convo(row, tokenizer):
  conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))  # reversed because context/5 in df comes first in conversation
  flatten = lambda l: [item for sublist in l for item in sublist]  # flattens each rown into a chunk
  conv = flatten(conv)
  return conv

class ConversationDataset(Dataset):
  def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):
    block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

    directory = args.cache_dir
    cached_features_file = os.path.join(
        directory, args.model_type + "_cached_lm_" + str(block_size)
    )

    if os.path.exists(cached_features_file) and not args.overwrite_cache:
      logger.info("Loading features from cached file %s", cached_features_file)
      with open(cached_features_file, "rb") as handle:
          self.examples = pickle.load(handle)
    else:
      logger.info("Creating features from dataset file at %s", directory)

      self.examples = []
      for _, row in df.iterrows():  # '_' is used as a convention to indicate an unused loop var
          conv = construct_convo(row, tokenizer)
          self.examples.append(conv)

      logger.info("Saving features into cached file %s", cached_features_file)
      with open(cached_features_file, "wb") as handle:
        pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

  def __len__(self):
    return len(self.examples)

  # Returns a specific example from the dataset at the given index as a torch tensor
  def __getitem__(self, item):
    return torch.tensor(self.examples[item], dtype=torch.long)

In [12]:
# Caching and storing of data/checkpoints
import re
import random
import glob  # UNIX style pathname pattern expression
import shutil  # high-level operations on files and collections of files
import numpy as np

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
   return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
   random.seed(args.seed)
   np.random.seed(args.seed)
   torch.manual_seed(args.seed)
   if args.n_gpu > 0:
       torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
   ordering_and_checkpoint_path = []

   glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

   for path in glob_checkpoints:
       if use_mtime:
           ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
       else:
           regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
           if regex_match and regex_match.groups():
               ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

   checkpoints_sorted = sorted(ordering_and_checkpoint_path)
   # Extract second element (checkpoint path) from each tuple in 'checkpoints_sorted'
   checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
   return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
   if not args.save_total_limit:
       return
   if args.save_total_limit <= 0:
       return

   # Check if we should delete older checkpoint(s)
   checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
   if len(checkpoints_sorted) <= args.save_total_limit:
       return

   number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
   checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
   for checkpoint in checkpoints_to_be_deleted:
       logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
       # Remove the entire directory associated with the checkpoint path
       shutil.rmtree(checkpoint)

In [13]:
from transformers import PreTrainedModel, AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm, trange

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()
    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
          # Pad a list of variable length Tensors
          # output will be in B X T X * format; B- Batch, T- Length of longest sequence
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    # RandomSampler, randomly samples elements from the training dataset without replacement
    # DistributedSampler, ensures that each process in the distributed setup sees a different portion of the trainig dataset
    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    # Dataloader enables the model to iterate over the dataset in batches during the trainig process
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )
    print("Batch size: ", args.train_batch_size)
    print("Data Loader length: ", len(train_dataloader))

    # t_total-total no. of training steps, len(train_dataloader)-no. of batches in training dataset,
    # args.gradient_accumulator_steps-how many gradient accumulation steps are performed before a parameter updates
    if args.max_steps > 0:
        t_total = args.max_steps
        # Calculate no. of epochs required to reach args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
      # total number of training steps required to complete the specified no. of epochs
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                print("Batch shape: ", batch.shape)
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [14]:
# Main runner
from transformers import AutoConfig

def main(df_trn, df_val):
    args = Args()

    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir='/content/'+args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir='/content/'+args.cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir='/content/'+args.cache_dir,
    )
    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)
        print(train_dataset)
        print("Train Dataset Length: ", len(train_dataset))
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelForCausalLM.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [15]:
class Args():
    def __init__(self):
        self.output_dir = 'TheoVonGPT-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'


args = Args()

In [16]:
main(trn_df, val_df)

Train Dataset Length:  25000
Batch size:  4
Data Loader length:  6250


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6250 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Iteration:   0%|          | 0/6250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6250 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

{'perplexity_': tensor(22.7376)}

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained('TheoVonGPT-small')

# Let's chat for 5 lines
for step in range(5):
  # encode the new user input, add the eos_token and return a tensor in PyTorch
  new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

  # append the new user input tokens to the chat history
  bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

  # generated a response while limiting the total chat history to 1000 tokens
  chat_history_ids = model.generate(
      bot_input_ids,
      max_length=1000,
      pad_token_id = tokenizer.eos_token_id,
      no_repeat_ngram_size=3,
      do_sample=True,
      top_k=100,
      top_p=0.7,
      temperature=0.8,
  )

  # pretty print last output tokens from bot
  print("Theo Von: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hey there


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Theo Von: [Music] and I'll sit and tell you my stories
>> User:tell me the craziest story theo


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Theo Von: [Applause] you know I'll tell you one story
>> User:how was your childhood


...

Theo Von: it was wild to be young and you know what I mean like it was wild
>> User:are you on drugs


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Theo Von: um I'm not sure I'm on drugs I'm
>> User:you definitely are on something


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Theo Von: !!!__ ] you're on drugs dude


### Push model to HuggingFace Hub

In [18]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [19]:
!git config --global user.email "7diwakarbasnet@gmail.com"
!git config --global user.name "Unspoiled-Egg"

In [23]:
MY_MODEL_NAME = "DialoGPT-small-TheoVon"
with open('HuggingFace-API-key.txt', 'rt') as f:
  HUGGINGFACE_API_KEY = f.read().strip()

In [25]:
model.push_to_hub(MY_MODEL_NAME, use_auth_token=HUGGINGFACE_API_KEY)
tokenizer.push_to_hub(MY_MODEL_NAME, use_auth_token=HUGGINGFACE_API_KEY)

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Unspoiled-Egg/DialoGPT-small-TheoVon/commit/6a2029de2341760be0d7394cbbef57d0f5108d81', commit_message='Upload tokenizer', commit_description='', oid='6a2029de2341760be0d7394cbbef57d0f5108d81', pr_url=None, pr_revision=None, pr_num=None)